<div style="background-color:#00000">
    <img src="https://fundacionsadosky.org.ar/wp-content/uploads/2022/08/logo.png" />
</div>
<div style="background-color:#03030a; margin:20px 45%">
    <img src="https://www.rfindustrial.com/wp-content/uploads/2023/04/cropped-1080x1080_Mesa-de-trabajo-1.png" />
</div>
<div style="background-color:#00000;">
    <img src="https://sinc.unl.edu.ar/wp-content/themes/sinci/img/sinc-logo.png" />
</div>

<p style="font-size: 30px">
    <strong>COPE - “Sistema inteligente de medición de nivel y control de velocidad de bombeo para pozos petrolíferos"</strong>
</p>

<p style="font-size: 20px">
    Objetivo del análisis: Realizar una experimentación para predecir el comportamiento del pozo.
</p>

25/09/2023

# Imports generales

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import sys
sys.path.append("../") # go to parent dir

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
# Set notebook mode to work in offline
pyo.init_notebook_mode(connected=True)

from src.data import utils
from src.methods import utils
from src.methods import base as base_methods

from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn_pandas import DataFrameMapper
from sklearn import preprocessing

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.base import BaseEstimator

from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import mean_absolute_error

import copy

class SameValueEstimator(BaseEstimator):
    def fit(self, X, y):
        return self
    
    def predict(self, X):
        return np.zeros(len(X))

RANDOM_SEED = 24
np.random.seed(RANDOM_SEED)

# Carga de datos

In [2]:
df = pd.read_csv('../data/raw/dinamometrias.csv')

In [3]:
df.head(2)

,fecha,pozo,porcentaje agua,presion casing,presion boca pozo,yacimiento,zona,carrera,gpm,tv,sv,cbe,tmax,vpmax,carga max fondo,carga min fondo,sobre recorrido,estiramiento,profundidad bomba,cuplas,produccion petroleo,profundidad gas,profundidad niveles,velocidad sonido,sumergencia efectiva,presion dinamica,presion interface
0,2022-10-14,F 281,98.45,0.5,4.5,DIADEMA,ZONA 2,167.99,7.94,15541,11275,15725,499222,1.1,5320,-3108,8.68,23.69,1925.74,18.2,0.1,171,1598,363,328,436,8
1,2022-10-20,H 129,92.85,0.0,6.5,DIADEMA,ZONA 4,168.77,5.04,18647,9080,16040,720122,0.7,10573,-2792,3.31,29.56,1454.49,142.6,6.1,1341,1367,388,88,117,2


In [4]:
df.describe()

,presion casing,presion boca pozo,carrera,gpm,tv,sv,cbe,tmax,vpmax,carga max fondo,carga min fondo,sobre recorrido,estiramiento,profundidad bomba,cuplas,produccion petroleo,profundidad gas,profundidad niveles,velocidad sonido,sumergencia efectiva,presion dinamica,presion interface
count,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000,2273.000000
mean,2.836252,11.040255,152.085631,5.665913,15125.555653,9613.488781,14959.559173,525910.671359,0.638583,6258.824901,-2798.278927,5.439534,24.658812,1626.684611,130.886450,5.942895,1230.312802,1354.754070,364.327761,271.737791,402.556973,47.991201
std,4.582152,7.396871,29.968334,1.872075,3683.317522,2564.983998,3317.775167,191646.861505,0.326733,2578.630607,1294.194567,3.784793,9.951332,361.211442,44.208121,8.279344,415.825745,386.975344,277.335485,287.542077,364.172842,74.407779
min,0.000000,0.000000,17.250000,1.700000,1805.000000,646.000000,0.000000,0.000000,0.100000,11.000000,-9577.000000,0.000000,0.970000,134.580000,8.500000,-11.400000,80.000000,82.000000,183.000000,1.000000,18.000000,0.000000
25%,0.000000,6.000000,143.780000,4.070000,12815.000000,8160.000000,13449.000000,397473.000000,0.400000,4399.000000,-3466.000000,2.570000,17.370000,1404.420000,103.700000,2.200000,973.000000,1101.000000,310.000000,65.000000,150.000000,2.000000
50%,0.100000,10.000000,167.560000,5.320000,15523.000000,9558.000000,15634.000000,550713.000000,0.600000,5951.000000,-2705.000000,4.690000,23.450000,1574.660000,127.000000,3.700000,1194.000000,1317.000000,355.000000,159.000000,277.000000,4.000000
75%,4.500000,15.000000,168.500000,7.090000,17983.000000,11549.000000,16565.000000,660738.000000,0.900000,7796.000000,-2003.000000,7.990000,31.310000,1932.900000,156.000000,6.800000,1468.000000,1665.000000,390.000000,368.000000,541.000000,76.000000
max,24.400000,56.000000,194.630000,11.680000,25081.000000,16983.000000,23083.000000,988476.000000,1.600000,16914.000000,7467.000000,23.090000,76.230000,2648.000000,269.400000,241.000000,2533.000000,2578.000000,7825.000000,1616.000000,2112.000000,391.000000


In [5]:
df['fecha'] = pd.to_datetime(df.fecha)
df = df.sort_values(by=['pozo', 'fecha'])
df.columns = [c.lower().replace(' ', '_') for c in df.columns]
df['porcentaje_agua'] = pd.to_numeric(df.porcentaje_agua, errors='coerce')

# Define useful things

In [6]:
def add_time_stamps_and_other_features(_df,
                                       cols_to_include,
                                       n=1,
                                       relative_diffs=True):
    _df = _df.sort_values(['pozo', 'fecha'])
    
    if n > 0:
        for i in range(n):
            shift = i + 1
            
            # Add diff in days to previous columns
            dias_column = f'dias_estudio_previo_{shift}'
            _df[dias_column] = _df.shift(shift - 1).fecha - _df.shift(shift).fecha

            # Add well previous study and filter mismatches.
            pozo_column = f'pozo_estudio_previo_{shift}'
            _df[pozo_column] = _df.shift(shift).pozo
            _df = _df[_df[pozo_column] == _df['pozo']].copy()

            for c in cols_to_include:
                if relative_diffs:
                    _df[f'diff_{c}_{shift}'] = (_df.shift(shift - 1)[c] - _df.shift(shift)[c]) / _df[c]
                else:
                    _df[f'diff_{c}_{shift}'] = (_df.shift(shift - 1)[c] - _df.shift(shift)[c])     

            _df[f'diff_fecha_{shift}'] = (_df.shift(shift)['fecha'] - _df.shift(shift - 1)['fecha']).dt.days

    _df['fecha_mes'] = _df.fecha.dt.month
    _df['zona_pozo'] = _df.pozo.str.replace(r'(\d+[.\d]*)', '', regex=True)
    _df.replace([np.inf], 100, inplace=True)
    _df.replace([-np.inf], -100, inplace=True)
    
    return _df

def add_label(_df, relative_diff=True):
    if relative_diff:
        _df['target'] = (_df.shift(-1).porcentaje_agua - _df.porcentaje_agua) / _df.porcentaje_agua
    else:
        _df['target'] = (_df.shift(-1).porcentaje_agua - _df.porcentaje_agua)

    _df['pozo_estudio_siguiente'] = _df.shift(-1).pozo
    _df['dias_estudio_proximo'] = (_df.shift(-1).fecha - _df.fecha).dt.days

    df_filtered = _df[_df.pozo == _df.pozo_estudio_siguiente].copy()

    df_filtered = df_filtered[df_filtered.target.abs() < 100]
    
    return df_filtered

In [7]:
cols_to_include = ['porcentaje_agua', 'presion_casing', 'presion_boca_pozo',
                   'tv', 'sv', 'cbe', 'tmax', 'vpmax', 'carga_max_fondo', 'carga_min_fondo',
                   'sobre_recorrido', 'estiramiento',
                   'profundidad_bomba', 'cuplas',
                   'produccion_petroleo', 'profundidad_gas', 'profundidad_niveles',
                   'velocidad_sonido', 'sumergencia_efectiva', 'presion_dinamica',
                   'presion_interface', 'gpm', 'carrera']

In [8]:
df = df[df.porcentaje_agua.notnull()]

In [9]:
model_instances = [
    ('DT', DecisionTreeRegressor(max_depth=2,
                                 min_samples_leaf=10,
                                 random_state=RANDOM_SEED)),
    ('Adaboost', AdaBoostRegressor(n_estimators=10,
                                   random_state=RANDOM_SEED)),
    ('GradientBoosting', GradientBoostingRegressor(n_estimators=10,
                                                   random_state=RANDOM_SEED)),
    ('MLP', MLPRegressor(hidden_layer_sizes=(100, 100),
                         max_iter=10000,
                         random_state=RANDOM_SEED)),
    ('SameValue', SameValueEstimator())
]

In [10]:
def run_experiment(_df, models, mapper):
    X = _df.copy()
    y = _df.target

    df_results = pd.DataFrame()

    for model_name, base_model_instance in models:
        model = copy.deepcopy(base_model_instance)

        pipe = Pipeline([
            ('mapper', mapper),
            ('model', model)])

        results = cross_validate(pipe, 
                                 X, y,
                                 cv=5,
                                 scoring='neg_mean_absolute_error',
                                 return_train_score=True)
        
        df_partial_results = pd.DataFrame(results, columns=results.keys())
        df_partial_results['method'] = model_name
        
        df_results = pd.concat([df_results, df_partial_results])
    
    print(df_results)
    
    print(df_results.groupby('method').test_score.mean())

# Evaluate results using n=1 (n is the number of previous studies to include as input features)

In [13]:
print(df.shape)
df_full = add_time_stamps_and_other_features(df.copy(), cols_to_include)
print(df_full.shape)
df_full = add_label(df_full)
print(df_full.shape)

for c in df_full.columns:
    if df_full[c].isnull().sum():
        df_full[c] = df_full[c].fillna(0)

(2270, 27)
(2130, 55)
(2005, 58)


In [14]:
one_hot_features = ['zona_pozo']
numerical_features = ['porcentaje_agua',
                      'presion_casing',
                      'presion_boca_pozo',
                      'profundidad_bomba',
                      'cuplas',
                      'produccion_petroleo',
                      'gpm',
                      'tv',
                      'sv',
                      'cbe',
                      'tmax',
                      'carga_max_fondo',
                      'carga_min_fondo',
                      'sobre_recorrido',
                      'estiramiento',
                      'carrera',
                      'profundidad_gas',
                      'profundidad_niveles',
                      'velocidad_sonido',
                      'sumergencia_efectiva',
                      'presion_dinamica',
                      'presion_interface',
                      'diff_fecha', 
                      'fecha_mes',
                      'dias_estudio_proximo']

mapper_inputs = []
numerical_preprocessing = preprocessing.MinMaxScaler

for c_base in numerical_features:
    related_cols = [c for c in df_full.columns if c_base in c]
    
    for c in related_cols:
        mapper_inputs.append(([c], numerical_preprocessing()))

for c in one_hot_features:
    mapper_inputs.append(([c], preprocessing.OneHotEncoder(handle_unknown='ignore')))

mapper = DataFrameMapper(mapper_inputs)

In [15]:
run_experiment(df_full, model_instances, mapper)

   fit_time  score_time  test_score  train_score            method
0  0.096423    0.047557   -0.192730    -0.170048                DT
1  0.081519    0.048734   -0.258897    -0.163311                DT
2  0.066130    0.033591   -0.310450    -0.166555                DT
3  0.060016    0.035776   -0.060038    -0.211328                DT
4  0.058897    0.029476   -0.149556    -0.193868                DT
0  0.180567    0.036220   -0.214709    -0.132568          Adaboost
1  0.185080    0.037313   -0.234063    -0.092586          Adaboost
2  0.203101    0.037154   -0.324284    -0.107114          Adaboost
3  0.185027    0.035222   -0.089547    -0.156143          Adaboost
4  0.180178    0.030461   -0.100767    -0.129050          Adaboost
0  0.258702    0.035556   -0.229609    -0.138260  GradientBoosting
1  0.261765    0.036274   -0.267557    -0.121446  GradientBoosting
2  0.238003    0.034622   -0.315082    -0.124835  GradientBoosting
3  0.261251    0.033886   -0.084480    -0.162907  GradientBoos

## Evaluate without relative diff in targets

In [19]:
print(df.shape)
df_full = add_time_stamps_and_other_features(df.copy(), cols_to_include)
print(df_full.shape)
df_full = add_label(df_full, relative_diff=False)
print(df_full.shape)

for c in df_full.columns:
    if df_full[c].isnull().sum():
        df_full[c] = df_full[c].fillna(0)

(2270, 27)
(2130, 55)
(2005, 58)


In [20]:
one_hot_features = ['zona_pozo']
numerical_features = ['porcentaje_agua',
                      'presion_casing',
                      'presion_boca_pozo',
                      'profundidad_bomba',
                      'cuplas',
                      'produccion_petroleo',
                      'gpm',
                      'tv',
                      'sv',
                      'cbe',
                      'tmax',
                      'carga_max_fondo',
                      'carga_min_fondo',
                      'sobre_recorrido',
                      'estiramiento',
                      'carrera',
                      'profundidad_gas',
                      'profundidad_niveles',
                      'velocidad_sonido',
                      'sumergencia_efectiva',
                      'presion_dinamica',
                      'presion_interface',
                      'diff_fecha', 
                      'fecha_mes',
                      'dias_estudio_proximo']

mapper_inputs = []
numerical_preprocessing = preprocessing.MinMaxScaler

for c_base in numerical_features:
    related_cols = [c for c in df_full.columns if c_base in c]
    
    for c in related_cols:
        mapper_inputs.append(([c], numerical_preprocessing()))

for c in one_hot_features:
    mapper_inputs.append(([c], preprocessing.OneHotEncoder(handle_unknown='ignore')))

mapper = DataFrameMapper(mapper_inputs)

In [23]:
run_experiment(df_full, model_instances, mapper)

   fit_time  score_time  test_score  train_score            method
0  0.072904    0.032734   -5.143360    -4.648292                DT
1  0.059818    0.034055   -7.047176    -4.293600                DT
2  0.061432    0.029184   -6.030764    -4.616590                DT
3  0.058394    0.030282   -3.946780    -5.213867                DT
4  0.055885    0.030376   -3.262234    -5.478209                DT
0  0.185094    0.031926   -5.374761    -4.666327          Adaboost
1  0.215958    0.030765   -6.796564    -4.184929          Adaboost
2  0.183117    0.031026   -7.121191    -5.368072          Adaboost
3  0.175133    0.030248   -3.990321    -4.900794          Adaboost
4  0.182579    0.034544   -3.675442    -5.528410          Adaboost
0  0.229591    0.029837   -5.183317    -4.347338  GradientBoosting
1  0.243941    0.029596   -7.104416    -4.006488  GradientBoosting
2  0.239519    0.031913   -5.820475    -4.285502  GradientBoosting
3  0.241948    0.029393   -3.929475    -4.714486  GradientBoos

# Evaluate results using n=2

In [24]:
print(df.shape)
df_full = add_time_stamps_and_other_features(df.copy(), cols_to_include, n=2)
print(df_full.shape)
df_full = add_label(df_full)
print(df_full.shape)

for c in df_full.columns:
    if df_full[c].isnull().sum():
        df_full[c] = df_full[c].fillna(0)

(2270, 27)
(1884, 81)
(1766, 84)


In [25]:
one_hot_features = ['zona_pozo']
numerical_features = ['porcentaje_agua',
                      'presion_casing',
                      'presion_boca_pozo',
                      'profundidad_bomba',
                      'cuplas',
                      'produccion_petroleo',
                      'gpm',
                      'tv',
                      'sv',
                      'cbe',
                      'tmax',
                      'carga_max_fondo',
                      'carga_min_fondo',
                      'sobre_recorrido',
                      'estiramiento',
                      'carrera',
                      'profundidad_gas',
                      'profundidad_niveles',
                      'velocidad_sonido',
                      'sumergencia_efectiva',
                      'presion_dinamica',
                      'presion_interface',
                      'diff_fecha', 
                      'fecha_mes',
                      'dias_estudio_proximo']

mapper_inputs = []
numerical_preprocessing = preprocessing.MinMaxScaler

for c_base in numerical_features:
    related_cols = [c for c in df_full.columns if c_base in c]
    
    for c in related_cols:
        mapper_inputs.append(([c], numerical_preprocessing()))

for c in one_hot_features:
    mapper_inputs.append(([c], preprocessing.OneHotEncoder(handle_unknown='ignore')))

mapper = DataFrameMapper(mapper_inputs)

In [26]:
run_experiment(df_full, model_instances, mapper)

   fit_time  score_time  test_score  train_score            method
0  0.095913    0.044919   -0.218543    -0.155417                DT
1  0.082586    0.046795   -0.255151    -0.162612                DT
2  0.081654    0.042305   -0.313264    -0.168850                DT
3  0.083953    0.047306   -0.058680    -0.210761                DT
4  0.080567    0.043020   -0.152800    -0.192458                DT
0  0.234732    0.045974   -0.213960    -0.138050          Adaboost
1  0.221121    0.043335   -0.251712    -0.116266          Adaboost
2  0.229225    0.044383   -0.224675    -0.118530          Adaboost
3  0.227557    0.049603   -0.092597    -0.159714          Adaboost
4  0.234115    0.043730   -0.105769    -0.130997          Adaboost
0  0.320792    0.042577   -0.226160    -0.136612  GradientBoosting
1  0.326404    0.048688   -0.261155    -0.129178  GradientBoosting
2  0.315753    0.042463   -0.270461    -0.125828  GradientBoosting
3  0.322010    0.047109   -0.075254    -0.160947  GradientBoos

# Evaluate results using n=3

In [27]:
print(df.shape)
df_full = add_time_stamps_and_other_features(df.copy(), cols_to_include, n=3)
print(df_full.shape)
df_full = add_label(df_full)
print(df_full.shape)

for c in df_full.columns:
    if df_full[c].isnull().sum():
        df_full[c] = df_full[c].fillna(0)

(2270, 27)
(1541, 107)
(1437, 110)


In [28]:
one_hot_features = ['zona_pozo']
numerical_features = ['porcentaje_agua',
                      'presion_casing',
                      'presion_boca_pozo',
                      'profundidad_bomba',
                      'cuplas',
                      'produccion_petroleo',
                      'gpm',
                      'tv',
                      'sv',
                      'cbe',
                      'tmax',
                      'carga_max_fondo',
                      'carga_min_fondo',
                      'sobre_recorrido',
                      'estiramiento',
                      'carrera',
                      'profundidad_gas',
                      'profundidad_niveles',
                      'velocidad_sonido',
                      'sumergencia_efectiva',
                      'presion_dinamica',
                      'presion_interface',
                      'diff_fecha', 
                      'fecha_mes',
                      'dias_estudio_proximo']

mapper_inputs = []
numerical_preprocessing = preprocessing.MinMaxScaler

for c_base in numerical_features:
    related_cols = [c for c in df_full.columns if c_base in c]
    
    for c in related_cols:
        mapper_inputs.append(([c], numerical_preprocessing()))

for c in one_hot_features:
    mapper_inputs.append(([c], preprocessing.OneHotEncoder(handle_unknown='ignore')))

mapper = DataFrameMapper(mapper_inputs)

In [29]:
run_experiment(df_full, model_instances, mapper)

   fit_time  score_time  test_score  train_score            method
0  0.114722    0.061577   -0.223209    -0.160832                DT
1  0.112792    0.064944   -0.250797    -0.171863                DT
2  0.099847    0.055633   -0.385204    -0.171374                DT
3  0.112263    0.061079   -0.040208    -0.222499                DT
4  0.106713    0.056478   -0.170521    -0.197893                DT
0  0.261696    0.058233   -0.238293    -0.125589          Adaboost
1  0.253331    0.059831   -0.269939    -0.132546          Adaboost
2  0.251761    0.056809   -0.274338    -0.152238          Adaboost
3  0.264746    0.057369   -0.133595    -0.191811          Adaboost
4  0.243811    0.067829   -0.130698    -0.136221          Adaboost
0  0.358358    0.060188   -0.239988    -0.142599  GradientBoosting
1  0.385785    0.060496   -0.267654    -0.130670  GradientBoosting
2  0.356460    0.055500   -0.294857    -0.119643  GradientBoosting
3  0.363940    0.057879   -0.066239    -0.161537  GradientBoos

### Testing not using feature calculation normalization for specific samples

In [30]:
print(df.shape)
df_full = add_time_stamps_and_other_features(df.copy(), cols_to_include, n=3, relative_diffs=False)
print(df_full.shape)
df_full = add_label(df_full)
print(df_full.shape)

for c in df_full.columns:
    if df_full[c].isnull().sum():
        df_full[c] = df_full[c].fillna(0)

(2270, 27)
(1541, 107)
(1437, 110)


In [31]:
one_hot_features = ['zona_pozo']
numerical_features = ['porcentaje_agua',
                      'presion_casing',
                      'presion_boca_pozo',
                      'profundidad_bomba',
                      'cuplas',
                      'produccion_petroleo',
                      'gpm',
                      'tv',
                      'sv',
                      'cbe',
                      'tmax',
                      'carga_max_fondo',
                      'carga_min_fondo',
                      'sobre_recorrido',
                      'estiramiento',
                      'carrera',
                      'profundidad_gas',
                      'profundidad_niveles',
                      'velocidad_sonido',
                      'sumergencia_efectiva',
                      'presion_dinamica',
                      'presion_interface',
                      'diff_fecha', 
                      'fecha_mes',
                      'dias_estudio_proximo']

mapper_inputs = []
numerical_preprocessing = preprocessing.MinMaxScaler

for c_base in numerical_features:
    related_cols = [c for c in df_full.columns if c_base in c]
    
    for c in related_cols:
        mapper_inputs.append(([c], numerical_preprocessing()))

for c in one_hot_features:
    mapper_inputs.append(([c], preprocessing.OneHotEncoder(handle_unknown='ignore')))

mapper = DataFrameMapper(mapper_inputs)

In [32]:
run_experiment(df_full, model_instances, mapper)

   fit_time  score_time  test_score  train_score            method
0  0.105080    0.064743   -0.231433    -0.153820                DT
1  0.108655    0.059929   -0.250797    -0.171863                DT
2  0.095612    0.055813   -0.385204    -0.171374                DT
3  0.098736    0.056762   -0.040208    -0.222499                DT
4  0.101308    0.058342   -0.167884    -0.197737                DT
0  0.244943    0.059683   -0.259586    -0.155694          Adaboost
1  0.233533    0.056784   -0.267263    -0.139818          Adaboost
2  0.218908    0.066142   -0.245702    -0.120472          Adaboost
3  0.227907    0.067787   -0.072219    -0.137197          Adaboost
4  0.219973    0.056998   -0.116926    -0.125122          Adaboost
0  0.302388    0.055265   -0.243645    -0.133044  GradientBoosting
1  0.299734    0.059094   -0.249500    -0.129485  GradientBoosting
2  0.286459    0.056415   -0.314952    -0.120380  GradientBoosting
3  0.287158    0.056012   -0.091056    -0.162325  GradientBoos

# Partial conclusions

Adaboost with n=1 and relative diffs is the best configuration.

# Evaluate filtering some noisy examples

In [33]:
df_full.dias_estudio_proximo.describe()

count    1437.000000
mean       34.631872
std        30.313592
min         0.000000
25%        28.000000
50%        30.000000
75%        35.000000
max       749.000000
Name: dias_estudio_proximo, dtype: float64

In [34]:
df_filtered = df_full[df_full.dias_estudio_proximo <= 35]
print(df_filtered.shape)
run_experiment(df_filtered, model_instances, mapper)

(1151, 110)
   fit_time  score_time  test_score  train_score            method
0  0.121896    0.071274   -0.247113    -0.176348                DT
1  0.103689    0.065631   -0.286110    -0.183879                DT
2  0.103839    0.060581   -0.419149    -0.195287                DT
3  0.114388    0.064120   -0.055895    -0.239830                DT
4  0.098035    0.063400   -0.134941    -0.220458                DT
0  0.201773    0.060722   -0.259520    -0.123848          Adaboost
1  0.201895    0.056492   -0.272869    -0.113224          Adaboost
2  0.227194    0.066940   -0.349987    -0.183707          Adaboost
3  0.198758    0.058117   -0.090978    -0.139117          Adaboost
4  0.201962    0.068801   -0.157092    -0.145660          Adaboost
0  0.277550    0.062278   -0.276380    -0.142038  GradientBoosting
1  0.272356    0.066982   -0.290751    -0.132294  GradientBoosting
2  0.268389    0.060912   -0.265556    -0.134846  GradientBoosting
3  0.272805    0.061910   -0.116387    -0.174108  

In [98]:
df_filtered = df_full[df_full.dias_estudio_proximo >= 35]
print(df_filtered.shape)
run_experiment(df_filtered, model_instances, mapper)

(538, 58)
   fit_time  score_time  test_score  train_score            method
0  0.088847    0.064563   -0.100483    -0.090252                DT
1  0.063091    0.037339   -0.210286    -0.068753                DT
2  0.061038    0.036376   -0.064924    -0.096855                DT
3  0.059660    0.042438   -0.100028    -0.092973                DT
4  0.057627    0.040512   -0.023677    -0.104846                DT
0  0.101645    0.036987   -0.098801    -0.061307          Adaboost
1  0.107612    0.040405   -0.199487    -0.054127          Adaboost
2  0.117027    0.038421   -0.069745    -0.077579          Adaboost
3  0.100299    0.039405   -0.104867    -0.074680          Adaboost
4  0.101406    0.043500   -0.026366    -0.086558          Adaboost
0  0.115829    0.044118   -0.117921    -0.075278  GradientBoosting
1  0.117134    0.036939   -0.207681    -0.058646  GradientBoosting
2  0.116565    0.037292   -0.063279    -0.087112  GradientBoosting
3  0.109109    0.034927   -0.095039    -0.082329  Gr

# Evaluate using some control variables in future as inputs

In [55]:
df_exp = add_time_stamps_and_other_features(df.copy(), cols_to_include)
df_exp['profundidad_bomba_nuevo_momento'] = df_exp.shift(-1).profundidad_bomba
df_exp = add_label(df_exp)

for c in df_exp.columns:
    if df_exp[c].isnull().sum():
        df_exp[c] = df_exp[c].fillna(0)

In [56]:
one_hot_features = ['zona_pozo']
numerical_features = ['porcentaje_agua',
                      'presion_casing',
                      'presion_boca_pozo',
                      'profundidad_bomba',
                      'cuplas',
                      'produccion_petroleo',
                      'gpm',
                      'tv',
                      'sv',
                      'cbe',
                      'tmax',
                      'carga_max_fondo',
                      'carga_min_fondo',
                      'sobre_recorrido',
                      'estiramiento',
                      'carrera',
                      'profundidad_gas',
                      'profundidad_niveles',
                      'velocidad_sonido',
                      'sumergencia_efectiva',
                      'presion_dinamica',
                      'presion_interface',
                      'diff_fecha', 
                      'fecha_mes',
                      'dias_estudio_proximo',
                      'profundidad_bomba_nuevo_momento']

mapper_inputs = []
numerical_preprocessing = preprocessing.MinMaxScaler

for c_base in numerical_features:
    related_cols = [c for c in df_exp.columns if c_base in c]
    
    for c in related_cols:
        mapper_inputs.append(([c], numerical_preprocessing()))

for c in one_hot_features:
    mapper_inputs.append(([c], preprocessing.OneHotEncoder(handle_unknown='ignore')))

mapper = DataFrameMapper(mapper_inputs)

In [57]:
run_experiment(df_exp, model_instances, mapper)

   fit_time  score_time  test_score  train_score            method
0  0.072663    0.039898   -0.192730    -0.170048                DT
1  0.066296    0.032051   -0.258897    -0.163311                DT
2  0.071558    0.031995   -0.310450    -0.166555                DT
3  0.062669    0.030904   -0.060038    -0.211328                DT
4  0.068507    0.033687   -0.149556    -0.193868                DT
0  0.182845    0.032770   -0.202588    -0.119684          Adaboost
1  0.193399    0.038861   -0.257590    -0.091697          Adaboost
2  0.181904    0.039871   -0.270804    -0.111015          Adaboost
3  0.176937    0.033907   -0.084763    -0.147542          Adaboost
4  0.187670    0.033492   -0.121263    -0.140095          Adaboost
0  0.251423    0.030845   -0.224040    -0.138260  GradientBoosting
1  0.252254    0.031772   -0.271997    -0.121446  GradientBoosting
2  0.255700    0.038162   -0.317218    -0.124835  GradientBoosting
3  0.263772    0.036239   -0.082062    -0.162907  GradientBoos

# Add stacking model (using baseline and Adaboost)

In [64]:
from sklearn.ensemble import StackingRegressor, RandomForestRegressor

In [82]:
from sklearn.base import RegressorMixin

In [83]:
class SameValueEstimator(BaseEstimator, RegressorMixin):
    def fit(self, X, y):
        return self
    
    def predict(self, X):
        return np.zeros(len(X))

In [92]:
base_estimators = [
    ('Adaboost', AdaBoostRegressor(n_estimators=10,
                                   random_state=RANDOM_SEED)),
    ('DT', DecisionTreeRegressor(max_depth=2,
                                 min_samples_leaf=10,
                                 random_state=RANDOM_SEED)),
    ('MLP', MLPRegressor(hidden_layer_sizes=(100, 100),
                         max_iter=10000,
                         random_state=RANDOM_SEED))
]

In [93]:
reg = StackingRegressor(
    estimators=base_estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=RANDOM_SEED)
)

In [94]:
model_instances_with_stacked = [
    ('DT', DecisionTreeRegressor(max_depth=2,
                                 min_samples_leaf=10,
                                 random_state=RANDOM_SEED)),
    ('Adaboost', AdaBoostRegressor(n_estimators=10,
                                   random_state=RANDOM_SEED)),
    ('GradientBoosting', GradientBoostingRegressor(n_estimators=10,
                                                   random_state=RANDOM_SEED)),
    ('MLP', MLPRegressor(hidden_layer_sizes=(100, 100),
                         max_iter=10000,
                         random_state=RANDOM_SEED)),
    ('SameValue', SameValueEstimator()),
    ('Stacked', reg)
]

In [95]:
print(df.shape)
df_full = add_time_stamps_and_other_features(df.copy(), cols_to_include)
print(df_full.shape)
df_full = add_label(df_full)
print(df_full.shape)

for c in df_full.columns:
    if df_full[c].isnull().sum():
        df_full[c] = df_full[c].fillna(0)

(2270, 27)
(2130, 55)
(2005, 58)


In [96]:
one_hot_features = ['zona_pozo']
numerical_features = ['porcentaje_agua',
                      'presion_casing',
                      'presion_boca_pozo',
                      'profundidad_bomba',
                      'cuplas',
                      'produccion_petroleo',
                      'gpm',
                      'tv',
                      'sv',
                      'cbe',
                      'tmax',
                      'carga_max_fondo',
                      'carga_min_fondo',
                      'sobre_recorrido',
                      'estiramiento',
                      'carrera',
                      'profundidad_gas',
                      'profundidad_niveles',
                      'velocidad_sonido',
                      'sumergencia_efectiva',
                      'presion_dinamica',
                      'presion_interface',
                      'diff_fecha', 
                      'fecha_mes',
                      'dias_estudio_proximo']

mapper_inputs = []
numerical_preprocessing = preprocessing.MinMaxScaler

for c_base in numerical_features:
    related_cols = [c for c in df_full.columns if c_base in c]
    
    for c in related_cols:
        mapper_inputs.append(([c], numerical_preprocessing()))

for c in one_hot_features:
    mapper_inputs.append(([c], preprocessing.OneHotEncoder(handle_unknown='ignore')))

mapper = DataFrameMapper(mapper_inputs)

In [97]:
run_experiment(df_full, model_instances_with_stacked, mapper)

    fit_time  score_time  test_score  train_score            method
0   0.078570    0.037554   -0.192730    -0.170048                DT
1   0.067706    0.034073   -0.258897    -0.163311                DT
2   0.070266    0.034860   -0.310450    -0.166555                DT
3   0.071270    0.036568   -0.060038    -0.211328                DT
4   0.083221    0.037470   -0.149556    -0.193868                DT
0   0.194511    0.039685   -0.214709    -0.132568          Adaboost
1   0.213566    0.044247   -0.234063    -0.092586          Adaboost
2   0.203035    0.036940   -0.324284    -0.107114          Adaboost
3   0.197429    0.040136   -0.089547    -0.156143          Adaboost
4   0.189813    0.039476   -0.100767    -0.129050          Adaboost
0   0.273312    0.040700   -0.229609    -0.138260  GradientBoosting
1   0.278035    0.038792   -0.267557    -0.121446  GradientBoosting
2   0.268221    0.035476   -0.315082    -0.124835  GradientBoosting
3   0.278432    0.036903   -0.084480    -0.16290

# Create one model per zone

In [18]:
print(df.shape)
df_full = add_time_stamps_and_other_features(df.copy(), cols_to_include)
print(df_full.shape)
df_full = add_label(df_full)
print(df_full.shape)

for c in df_full.columns:
    if df_full[c].isnull().sum():
        df_full[c] = df_full[c].fillna(0)

(2270, 27)
(2130, 55)
(2005, 58)


In [28]:
df_full['zona'] = df_full.zona.str.replace('-', ' ').str.replace('  ', ' ').str.upper()

In [31]:
numerical_features = ['porcentaje_agua',
                      'presion_casing',
                      'presion_boca_pozo',
                      'profundidad_bomba',
                      'cuplas',
                      'produccion_petroleo',
                      'gpm',
                      'tv',
                      'sv',
                      'cbe',
                      'tmax',
                      'carga_max_fondo',
                      'carga_min_fondo',
                      'sobre_recorrido',
                      'estiramiento',
                      'carrera',
                      'profundidad_gas',
                      'profundidad_niveles',
                      'velocidad_sonido',
                      'sumergencia_efectiva',
                      'presion_dinamica',
                      'presion_interface',
                      'diff_fecha', 
                      'fecha_mes',
                      'dias_estudio_proximo']

mapper_inputs = []
numerical_preprocessing = preprocessing.MinMaxScaler

for c_base in numerical_features:
    related_cols = [c for c in df_full.columns if c_base in c]
    
    for c in related_cols:
        mapper_inputs.append(([c], numerical_preprocessing()))

mapper = DataFrameMapper(mapper_inputs)

In [37]:
zones = [z for z in df_full.zona.unique() if 'ZONA' in z]

df_results = pd.DataFrame()

for zone in zones:
    X = df_full[df_full.zona == zone].copy()
    y = df_full[df_full.zona == zone].target.copy()

    for model_name, base_model_instance in model_instances:
        model = copy.deepcopy(base_model_instance)

        pipe = Pipeline([
            ('mapper', mapper),
            ('model', model)])

        results = cross_validate(pipe, 
                                 X, y,
                                 cv=5,
                                 scoring='neg_mean_absolute_error',
                                 return_train_score=True)

        df_partial_results = pd.DataFrame(results, columns=results.keys())
        df_partial_results['method'] = model_name
        df_partial_results['zone'] = zone

        df_results = pd.concat([df_results, df_partial_results])

print(df_results)

print(df_results.groupby('method').test_score.mean())

    fit_time  score_time  test_score  train_score     method    zone
0   0.070922    0.044851   -0.182140    -0.291836         DT  ZONA 6
1   0.054231    0.032724   -0.194012    -0.294203         DT  ZONA 6
2   0.061381    0.033965   -0.307458    -0.269590         DT  ZONA 6
3   0.051377    0.030504   -0.360978    -0.249772         DT  ZONA 6
4   0.050096    0.032245   -0.665586    -0.264400         DT  ZONA 6
..       ...         ...         ...          ...        ...     ...
0   0.047306    0.027512   -0.071935    -0.017123  SameValue  ZONA 7
1   0.043624    0.027401   -0.016883    -0.030886  SameValue  ZONA 7
2   0.043715    0.026842   -0.024103    -0.029081  SameValue  ZONA 7
3   0.053611    0.028184   -0.013463    -0.031741  SameValue  ZONA 7
4   0.056106    0.027197   -0.014043    -0.031596  SameValue  ZONA 7

[150 rows x 6 columns]
method
Adaboost           -0.161300
DT                 -0.171981
GradientBoosting   -0.176766
MLP                -0.325165
SameValue          -0.122

# Add near well features

In [11]:
print(df.shape)
df_full = add_time_stamps_and_other_features(df.copy(), cols_to_include)
print(df_full.shape)
df_full = add_label(df_full)
print(df_full.shape)

for c in df_full.columns:
    if df_full[c].isnull().sum():
        df_full[c] = df_full[c].fillna(0)

(2270, 27)
(2130, 55)
(2005, 58)


In [12]:
df_well_locations = pd.read_csv('../data/external/well_locations.csv')

In [104]:
def get_nearest_well(well_id, df_wells, valid_wells):
    _df = df_wells[df_wells.well_id.str.replace('-', ' ') == well_id]
    
    if len(_df) > 0:
        well_x = _df.coor_x.abs().values[0]
        well_y = _df.coor_y.abs().values[0]
        mask = df_wells.well_id.str.replace('-', ' ').isin(valid_wells)
        df_wells['distance'] = np.sqrt((df_wells[mask].coor_x.abs() - well_x) ** 2 + (df_wells[mask].coor_y.abs() - well_y) ** 2)
        
        return df_wells.sort_values('distance').well_id.values[1]

    return None

In [129]:
def add_value_from_nearest_well(df, column):
    new_column_name = f'{column}_pozo_cercano'
    df[new_column_name] = None

    for well in df.pozo.unique():
        _df = df[df.pozo == well]
        near_well = get_nearest_well(well, df_well_locations, df.pozo.unique())
        if near_well:
            near_well = near_well.replace('-', ' ')
        
        for index, r in _df.iterrows():
            _df2 = df[df.pozo == near_well].copy()
            _df2['date_distance'] = r['fecha'] - _df2.fecha
            _df2 = _df2[_df2.date_distance.dt.days > 0].sort_values('date_distance')[column]
            if len(_df2) > 0:
                most_recent_value = _df2.values[0]
            else:
                most_recent_value = None
            
            mask1 = df.pozo == r['pozo']
            mask2 = df.fecha == r['fecha']
            df.loc[mask1 & mask2, f'{column}_pozo_cercano'] = most_recent_value
    
    return df

## Test only adding porcentaje_agua from nearest well

In [130]:
_df_full = add_value_from_nearest_well(df_full, 'porcentaje_agua')

In [131]:
_df_full[_df_full.porcentaje_agua_pozo_cercano.notnull()].shape

(816, 59)

In [132]:
one_hot_features = ['zona_pozo']
numerical_features = ['porcentaje_agua',
                      'presion_casing',
                      'presion_boca_pozo',
                      'profundidad_bomba',
                      'cuplas',
                      'produccion_petroleo',
                      'gpm',
                      'tv',
                      'sv',
                      'cbe',
                      'tmax',
                      'carga_max_fondo',
                      'carga_min_fondo',
                      'sobre_recorrido',
                      'estiramiento',
                      'carrera',
                      'profundidad_gas',
                      'profundidad_niveles',
                      'velocidad_sonido',
                      'sumergencia_efectiva',
                      'presion_dinamica',
                      'presion_interface',
                      'diff_fecha', 
                      'fecha_mes',
                      'dias_estudio_proximo']

mapper_inputs = []
numerical_preprocessing = preprocessing.MinMaxScaler

for c_base in numerical_features:
    related_cols = [c for c in df_full.columns if c_base in c]
    
    for c in related_cols:
        mapper_inputs.append(([c], numerical_preprocessing()))

for c in one_hot_features:
    mapper_inputs.append(([c], preprocessing.OneHotEncoder(handle_unknown='ignore')))

mapper = DataFrameMapper(mapper_inputs)

In [134]:
_df_full.porcentaje_agua_pozo_cercano.fillna(-1, inplace=True)

In [135]:
run_experiment(_df_full, model_instances, mapper)

   fit_time  score_time  test_score  train_score            method
0  0.081622    0.038659   -0.192730    -0.170048                DT
1  0.069850    0.034776   -0.258897    -0.163311                DT
2  0.070741    0.035094   -0.310450    -0.166555                DT
3  0.064961    0.032558   -0.060038    -0.211328                DT
4  0.061966    0.032674   -0.149556    -0.193868                DT
0  0.186217    0.035099   -0.203183    -0.118286          Adaboost
1  0.185023    0.033571   -0.288692    -0.124671          Adaboost
2  0.182009    0.035485   -0.279264    -0.107175          Adaboost
3  0.174165    0.034201   -0.087973    -0.156208          Adaboost
4  0.170174    0.034875   -0.123235    -0.144914          Adaboost
0  0.261930    0.032577   -0.224611    -0.138260  GradientBoosting
1  0.266898    0.035227   -0.269990    -0.121446  GradientBoosting
2  0.270760    0.033993   -0.312089    -0.124835  GradientBoosting
3  0.265841    0.032312   -0.082062    -0.162907  GradientBoos

## Add all variables

In [140]:
numerical_features

['porcentaje_agua',
 'presion_casing',
 'presion_boca_pozo',
 'profundidad_bomba',
 'cuplas',
 'produccion_petroleo',
 'gpm',
 'tv',
 'sv',
 'cbe',
 'tmax',
 'carga_max_fondo',
 'carga_min_fondo',
 'sobre_recorrido',
 'estiramiento',
 'carrera',
 'profundidad_gas',
 'profundidad_niveles',
 'velocidad_sonido',
 'sumergencia_efectiva',
 'presion_dinamica',
 'presion_interface',
 'diff_fecha',
 'fecha_mes',
 'dias_estudio_proximo']

In [ ]:
df_exp = df_full.copy()

cols_to_extract = ['porcentaje_agua',
 'presion_casing',
 'presion_boca_pozo',
 'profundidad_bomba',
 'cuplas',
 'produccion_petroleo',
 'gpm',
 'tv',
 'sv',
 'cbe',
 'tmax',
 'carga_max_fondo',
 'carga_min_fondo',
 'sobre_recorrido',
 'estiramiento',
 'carrera',
 'profundidad_gas',
 'profundidad_niveles',
 'velocidad_sonido',
 'sumergencia_efectiva',
 'presion_dinamica',
 'presion_interface']

for c in cols_to_extract:
    df_exp = add_value_from_nearest_well(df_exp, c)

In [ ]:
df_exp.fillna(-100, inplace=True)

In [154]:
one_hot_features = ['zona_pozo']
numerical_features = ['porcentaje_agua',
                      'presion_casing',
                      'presion_boca_pozo',
                      'profundidad_bomba',
                      'cuplas',
                      'produccion_petroleo',
                      'gpm',
                      'tv',
                      'sv',
                      'cbe',
                      'tmax',
                      'carga_max_fondo',
                      'carga_min_fondo',
                      'sobre_recorrido',
                      'estiramiento',
                      'carrera',
                      'profundidad_gas',
                      'profundidad_niveles',
                      'velocidad_sonido',
                      'sumergencia_efectiva',
                      'presion_dinamica',
                      'presion_interface',
                      'diff_fecha', 
                      'fecha_mes',
                      'dias_estudio_proximo']

mapper_inputs = []
numerical_preprocessing = preprocessing.MinMaxScaler

for c_base in numerical_features:
    related_cols = [c for c in df_exp.columns if c_base in c]
    
    for c in related_cols:
        mapper_inputs.append(([c], numerical_preprocessing()))

for c in one_hot_features:
    mapper_inputs.append(([c], preprocessing.OneHotEncoder(handle_unknown='ignore')))

mapper = DataFrameMapper(mapper_inputs)

In [156]:
run_experiment(df_exp, model_instances, mapper)

   fit_time  score_time  test_score  train_score            method
0  0.119614    0.053640   -0.192730    -0.170048                DT
1  0.094937    0.045754   -0.258897    -0.163311                DT
2  0.091695    0.050770   -0.310450    -0.166555                DT
3  0.095129    0.050264   -0.060038    -0.211328                DT
4  0.095631    0.049182   -0.149556    -0.193868                DT
0  0.243345    0.048627   -0.199135    -0.123988          Adaboost
1  0.213233    0.048727   -0.260773    -0.089212          Adaboost
2  0.223391    0.047813   -0.290654    -0.111478          Adaboost
3  0.222834    0.052933   -0.092666    -0.165160          Adaboost
4  0.218532    0.047735   -0.126363    -0.147023          Adaboost
0  0.292730    0.048441   -0.223911    -0.138370  GradientBoosting
1  0.302299    0.045928   -0.269102    -0.121446  GradientBoosting
2  0.309858    0.046269   -0.293034    -0.124835  GradientBoosting
3  0.307172    0.051820   -0.082062    -0.162907  GradientBoos